In [1]:
#csvファイルの読み込み
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
df_train = pd.read_csv(r"C:\Users\eikik\python\python_dat\equity-post-HCT-survival-predictions\train_preprocessed.csv")

In [2]:
#データの確認
df_train

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,target
0,N/A - non-malignant indication,No,Poor,No,-0.001785,-0.002142,No TBI,No,0.754491,Bone marrow,...,No,Permissive mismatched,Unrelated,"N/A, Mel not given",0.751129,No,0.675826,No,0.755836,-0.834709
1,Intermediate,No,Intermediate,No,0.595336,0.804529,"TBI +- Other, >cGy",No,0.754491,Peripheral blood,...,No,Permissive mismatched,Related,"N/A, Mel not given",0.751129,No,0.675826,Yes,0.755836,1.367042
2,N/A - non-malignant indication,No,Poor,No,0.595336,0.804529,No TBI,No,0.754491,Bone marrow,...,No,Permissive mismatched,Related,"N/A, Mel not given",0.751129,No,0.675826,No,0.755836,-0.813559
3,High,No,Intermediate,No,0.595336,0.804529,No TBI,No,0.754491,Bone marrow,...,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",0.751129,No,0.675826,No,0.755836,-0.846174
4,High,No,Poor,No,0.595336,0.804529,No TBI,No,0.754491,Peripheral blood,...,No,Permissive mismatched,Related,MEL,0.751129,No,0.675826,No,0.755836,-0.800830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,Intermediate - TED AML case <missing cytogenetics,No,Favorable,No,0.595336,0.804529,No TBI,No,0.754491,Peripheral blood,...,No,Bi-directional non-permissive,Related,"N/A, Mel not given",0.751129,No,0.675826,No,0.755836,-0.811172
28796,High,No,Poor,Yes,-1.931355,-2.058671,No TBI,No,-0.124932,Peripheral blood,...,No,GvH non-permissive,Related,"N/A, Mel not given",-0.616506,Yes,-1.630964,Yes,-0.389694,1.241045
28797,TBD cytogenetics,No,Poor,No,0.595336,0.804529,No TBI,No,0.754491,Peripheral blood,...,No,GvH non-permissive,Unrelated,"N/A, Mel not given",0.751129,No,0.675826,No,0.755836,-0.820817
28798,N/A - non-malignant indication,No,Poor,No,-1.931355,-2.058671,No TBI,No,-1.883777,Peripheral blood,...,No,Permissive mismatched,Related,MEL,-1.984141,No,-1.630964,No,-2.107990,-0.836307


これもだめでした。

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from lifelines.utils import concordance_index

# 特徴量とターゲットを分割
X = df_train.drop(columns=["target"])
y = df_train["target"]

# カテゴリ列のワンホットエンコーディング
categorical_columns = X.select_dtypes(include=["object"]).columns
encoder = OneHotEncoder(sparse_output=False, drop="first")
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_columns]),
                         columns=encoder.get_feature_names_out(categorical_columns))

# 数値データを維持
numerical_columns = X.select_dtypes(include=["float64", "int64"]).columns
X_numerical = X[numerical_columns]

# エンコードされたカテゴリデータと数値データを結合
X_preprocessed = pd.concat([X_numerical.reset_index(drop=True), X_encoded.reset_index(drop=True)], axis=1)

# 標準化
scaler = StandardScaler()
X_preprocessed = pd.DataFrame(scaler.fit_transform(X_preprocessed), columns=X_preprocessed.columns)

# 訓練データと検証データに分割
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# テンソルに変換
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# データローダーの作成
batch_size = 256
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# ニューラルネットワークモデルの定義
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# モデル、損失関数、オプティマイザの設定
input_size = X_train.shape[1]
model = NeuralNet(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# 早期終了のパラメータ
epochs = 200
patience = 10
best_c_index = -np.inf
patience_counter = 0

# モデルの訓練
for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # 検証データでの評価
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor).squeeze().numpy()
        c_index = concordance_index(y_test, -predictions)

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, C-index: {c_index:.4f}")

    # 早期終了のチェック
    if c_index > best_c_index:
        best_c_index = c_index
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break

print(f"Best C-index: {best_c_index:.4f}")


Epoch [1/200], Loss: 81.4978, C-index: 0.3784
Epoch [2/200], Loss: 75.0590, C-index: 0.3764
Epoch [3/200], Loss: 72.6704, C-index: 0.3725
Epoch [4/200], Loss: 70.6232, C-index: 0.3740
Epoch [5/200], Loss: 68.0755, C-index: 0.3782
Epoch [6/200], Loss: 65.3971, C-index: 0.3812
Epoch [7/200], Loss: 62.7122, C-index: 0.3807
Epoch [8/200], Loss: 60.0438, C-index: 0.3819
Epoch [9/200], Loss: 57.7149, C-index: 0.3840
Epoch [10/200], Loss: 54.0593, C-index: 0.3909
Epoch [11/200], Loss: 51.6016, C-index: 0.3877
Epoch [12/200], Loss: 48.9485, C-index: 0.3955
Epoch [13/200], Loss: 46.3014, C-index: 0.3949
Epoch [14/200], Loss: 44.2730, C-index: 0.3992
Epoch [15/200], Loss: 41.7782, C-index: 0.3984
Epoch [16/200], Loss: 40.0482, C-index: 0.3971
Epoch [17/200], Loss: 37.9994, C-index: 0.4029
Epoch [18/200], Loss: 35.9035, C-index: 0.4003
Epoch [19/200], Loss: 34.3061, C-index: 0.3987
Epoch [20/200], Loss: 32.7886, C-index: 0.4041
Epoch [21/200], Loss: 31.3262, C-index: 0.4064
Epoch [22/200], Loss: 